# Schalldruck des diffusen, reflektierten Schallfeldes $L_r$

In [43]:
import math
import numpy as np


volume = 5*4*2 # Abschaetzung eines normal grossen Raumes
surface = [20,20,10,10,8,8] # Abschaetzung eines normal grossen Raumes
# Testwerte fuer die diffusen Absorptionsgrade
alpha_d = {'125 Hz': [0.1, 0.1,0.1, 0.1,0.1, 0.1], '250 Hz': [0.1, 0.1,0.1, 0.1,0.1, 0.1], 
           '500 Hz': [0.1, 0.1,0.1, 0.1,0.1, 0.2], '1 kHz': [0.1, 0.1, 0.1, 0.2,  0.1, 0.1], 
           '2 kHz': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1], '4 kHz': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]} # 
power = 7 * 10**(-7) # Mittelwert Unterhaltungssprache nach Tabelle 5.2 im Skript technische Akustik
distance = 1

class Raum: 
    def __init__(self, volume, surface, alpha_d, power, distance):
        self.input = {'Volume': volume, 'Surface': surface, 'Absorption coefficient': alpha_d,
                       'Source pwoer': power, 'Distance source <-> receiver': distance}
        self.volume = volume
        self.surface = surface
        self.alpha_d = alpha_d
        self.power = power
        self.distance = distance
        
    
    def Eq_A(self, surface, alpha_d):
        # surface als list mit m^2 der einzelnen Waende
        # alpha_d als dictionary mit Oktavbandfrequenzen als key und Liste der diffusen Absorptionsgrade pro Wand als value   
        A = {'125 Hz':0 , '250 Hz':0 , '500 Hz':0 , '1 kHz':0, '2 kHz':0 , '4 kHz':0 }
        j = 0
        for k in range(len(surface)):
            for i in A:
                X = alpha_d[i]
                A[i] = A[i] + surface[j] * X[j]
            j =+ 1
        return A
   
    def Nachhallzeit(self, volume, surface, alpha_d):
        RT = {'125 Hz':0 , '250 Hz':0 , '500 Hz':0 , '1 kHz':0, '2 kHz':0 , '4 kHz':0 }
        A = self.Eq_A(surface, alpha_d)
        for i in A:
            RT[i] = (volume / A[i]) * 0.161
        return RT
    
    def Level_diffuse(self, surface, alpha_d, power):
        L_R = {'125 Hz':0 , '250 Hz':0 , '500 Hz':0 , '1 kHz':0, '2 kHz':0 , '4 kHz':0 }
        A = self.Eq_A(surface, alpha_d)
        for i in alpha_d:
            L_R[i] = 10 * math.log10(power / 10**(-12)) - 10 * math.log10(A[i]) + 6
        return L_R
    
    def Level_direct(self, power, distance):
        L_D = 10 * math.log10(power / 10**(-12)) - 10 * math.log10(4 * np.pi * distance**2)
        return L_D

    def Hallradius(self, surface, alpha_d, power):
        distance = 10**(-10)
        L_R = self.Level_diffuse(surface, alpha_d, power)
        L_D = self.Level_direct(power, distance)

        L_Rm = 0 # Initalisierung fuer Mittelwert des reflektierten Schalldruckpegels ueber alle Oktavbaender
        # Mittelwertbildung des reflektierten Schalldruckpegels
        for i in L_R:
            L_Rm = L_Rm + L_R[i] 
        L_Rm = L_Rm / len(L_R)
        # Sobald Mittelwert des reflektierten Schalldruckpegels
        # groesser als direkter Schalldruckpegel ist wird Hallradius ausgegeben
        while L_Rm < L_D:
            distance = distance + 0.01
            L_D = self.Level_direct(power, distance)            
        return distance
    
    def Sprachverständlichkeit(self, surface, alpha_d, power, distance):
        L_R = self.Level_diffuse(surface, alpha_d, power)
        L_Rm = 0
        for i in L_R:
            L_Rm = L_Rm + L_R[i] 
        L_Rm = L_Rm / len(L_R)

        SV = self.Level_direct(power, distance) - L_Rm
        return SV
    
Raum_1 = Raum(volume , surface, alpha_d, power, distance)
Raum_1.input

# Raum_1.surface[1] = 5 # Zuweisung neuer Wandgroesse wird nicht gespeichert. Warum nochmal? 
#                       # Wichtig fuer neue vorgeschlagene Konfigurationen
# Raum_1.surface

A = Raum_1.Eq_A(surface, alpha_d)
RT = Raum_1.Nachhallzeit(volume, surface, alpha_d)
L_R = Raum_1.Level_diffuse(surface, alpha_d, power)


L_D = Raum_1.Level_direct(power, distance)
print(L_D)

HR = Raum_1.Hallradius(surface, alpha_d, power)
print(RT)
print(L_R)
SV = Raum_1.Sprachverständlichkeit(surface, alpha_d, power, distance)
print(SV)


47.45888175992161
{'125 Hz': 0.05366666666666667, '250 Hz': 0.21466666666666667, '500 Hz': 0.5366666666666667, '1 kHz': 0.5366666666666667, '2 kHz': 0.06313725490196079, '4 kHz': 0.06313725490196079}
{'125 Hz': 43.65916793966632, '250 Hz': 49.67976785294594, '500 Hz': 53.65916793966632, '1 kHz': 53.65916793966632, '2 kHz': 44.36497868252339, '4 kHz': 44.36497868252339}
-0.7723230795770064
